# Data Preparation

This is a template notebook for data preparation.

Author: {{ cookiecutter.author_name }}
Created: {{ cookiecutter.timestamp }}


In [0]:
# Link to project experiments folder hypothesis_experiment_learnings.board (refresh and hit enter on this line to see the link)

## How to use the notebook

The following cells:
- import the data,
- perform Data Structuring,
    - Merge dataframes
    - Pivot dataframes
    - Separate Mixed Populations
- perform Data Cleansing,
    - Standardise data types, row/column names, and datetimes (if applicable)
    - Delete or augment missing data/rows
    - Plot percentile based outlier detection
- perform Data Transformation and Enrichment,
    - Encoding (Label/One-hot)
    - Aggregating
    - Binning
    - Regularisation
    - Dimensionality Reduction
    - Feature Selection
    - Consecutive usable time ranges (for time-series)

By default, the notebook is set up to run with an example (Daily Delhi Climate). To see how it works, run the notebook without changing the code.

For your project, adjust the code in the linked cells with your methods and then execute cells in the order you wish to process your data.

Please refer to data_preparation_guide.board for detailed instructions. The headers in this notebook follow the cards on the board.

### Imports and general setup

In [0]:
import numpy as np
import pandas as pd

from modules.modules import *

## 2. Import the Data

#### 2.1 Read the Data

In [0]:
# Default example file
path = 'https://raw.githubusercontent.com/erium/halerium-example-data/main/data_preparation/DailyDelhiClimateTrainMissing.csv'

# <halerium id="d9a0fe42-e917-472c-bac8-00390a3874ea">
filepaths = [path] # Multiple filepaths
sheet = 0 # Sheet number for Excel
sep = " " # Separator for text files
datetime_col = 'date' # Column name for datetime column
# </halerium id="d9a0fe42-e917-472c-bac8-00390a3874ea">


#### 2.2 Merge the Data

In [0]:
# <halerium id="797fba5f-2179-4412-aa77-bb7f4026f0a0">
join = 'vertical' # 'vertical', 'inner' or 'outer' (if >1 file)
# </halerium id="797fba5f-2179-4412-aa77-bb7f4026f0a0">


Import and merge the raw data

In [0]:
df = Importer.import_data(filepaths, sheet, sep, datetime_col, join)

Visualise the raw imported data

In [0]:
df

In [0]:
# <halerium id="c8f3911e-b463-4384-a35b-5c975c3b5eb5">
Plotter.plot_line(df)
# </halerium id="c8f3911e-b463-4384-a35b-5c975c3b5eb5">


## 3. Data Structuring

#### 3.1 Pivot the dataframe

In [0]:
# <halerium id="02e67e29-bbd9-4cd9-8231-7aa456c69291">
pivot = False
# </halerium id="02e67e29-bbd9-4cd9-8231-7aa456c69291">

if pivot:
    df = df.T
df

#### 3.2 Separate Mixed Populations

In [0]:
# <halerium id="6c0ed4e8-8d70-48ad-94b1-f74dcbb0167d">
Plotter.plot_line(df)
# </halerium id="6c0ed4e8-8d70-48ad-94b1-f74dcbb0167d">


In [0]:
# <halerium id="29a9c302-9106-46d9-b1b7-91510a813220">
population_indexes = [("Full", ('2013-01-01', '2017-01-01'))] # Tuples of (Population name, (index ranges)) for different populations; example data contains only 1 population
label_or_position = 'label' # 'label' for label indexes or 'position' for integer 0-based position indexing
# </halerium id="29a9c302-9106-46d9-b1b7-91510a813220">

populations = PopulationSeparator.separate_populations(df, population_indexes, label_or_position)
list(populations.keys())

Select population

In [0]:
df = populations["Full"]
df

## 4. Data Cleansing

#### 4.1 Stadardization

Data types

In [0]:
# <halerium id="70169b7a-842e-4285-9d13-48f6019ec3a3">
columns = list(df.columns) # Array of columns you wish to change 
datatype = 'float' # Pandas dtype: 'int', 'float', 'boolean', 'string'...
# </halerium id="70169b7a-842e-4285-9d13-48f6019ec3a3">

df = Standardizer.standardize_datatype(df, columns, datatype)
df

Column names

In [0]:
# <halerium id="70169b7a-842e-4285-9d13-48f6019ec3a3">
def rename_col_function(col_name): # Define a function to modify the column name
    col_name = col_name.replace("_", "")
    col_name = col_name.lower()
    return col_name
# </halerium id="70169b7a-842e-4285-9d13-48f6019ec3a3">

df = Standardizer.standardize_column_names(df, rename_col_function)
df

#### 4.2 Missing values

In [0]:
# <halerium id="208b9d84-2b3d-4a98-aa41-b860f2190ff1">
method = 'mean' # 'delete', 'zero', 'custom', 'mean', 'median', 'mode', 'linear', 'knn', 'interpolate', 'encode'
# <halerium id="208b9d84-2b3d-4a98-aa41-b860f2190ff1">
# </halerium id="208b9d84-2b3d-4a98-aa41-b860f2190ff1">
custom_value = 'None' # Custom value if method == 'custom'
# </halerium id="208b9d84-2b3d-4a98-aa41-b860f2190ff1">
# Note: 'mean', 'median', and 'interpolate' do not work on missing categorical values (implies cardinality)
df = MissingValues.handle_missing(df, method, custom_value)
df

#### 4.3 Outliers

In [0]:
Plotter.plot_box(df)

In [0]:
# <halerium id="c8f3911e-b463-4384-a35b-5c975c3b5eb5">
percentile = 0.99 # Specify outlier detection percentile threshold
upper_or_lower = 'upper' # 'upper' or 'lower' points exceeding percentile
features = df.columns  # Specify columns to detect outliers on
# </halerium id="c8f3911e-b463-4384-a35b-5c975c3b5eb5">

outliers = Outliers.find_outliers(percentile, upper_or_lower, df[features])
Plotter.show_outliers(df, outliers)
outliers

Remove outliers

In [0]:
df = Outliers.remove_outliers(df, outliers)
df

In [0]:
Plotter.plot_line(df)

In [0]:
Plotter.plot_box(df)

## 5. Data Transformation and Enrichment

#### 5.1 Encoding

In [0]:
# <halerium id="6c0ed4e8-8d70-48ad-94b1-f74dcbb0167d">
one_hot_threshold = 10 # Threshold to switch to label encoding
# </halerium id="6c0ed4e8-8d70-48ad-94b1-f74dcbb0167d">
df = Encoder.encode(df, one_hot_threshold)
df

#### 5.2 Transformation and Enrichment

Operations on columns
Note: Will modify existing columns

In [0]:
# <halerium id="9330ed5e-3380-4b17-b8e8-7c7545a6f1bd">
single_cols = ["meantemp"] # Columns to apply the function on
def single_cols_function(value): # Function
    return value * 2
# </halerium id="9330ed5e-3380-4b17-b8e8-7c7545a6f1bd">

df = Transformer.single_column_operation(df, single_cols, single_cols_function)
df

Operations between columns

In [0]:
# <halerium id="9330ed5e-3380-4b17-b8e8-7c7545a6f1bd">
multi_cols = ['meanpressure', 'meantemp', 'humidity'] # Order of columns
multi_cols_operations = ['divide', 'add'] # Order of operations between each column, 'add', 'subtract', 'multiply', 'divide
result_col_name = 'result'
# </halerium id="9330ed5e-3380-4b17-b8e8-7c7545a6f1bd">

df = Transformer.multi_column_operation(df, multi_cols, multi_cols_operations, result_col_name)
df

Normalisation

In [0]:
# <halerium id="9330ed5e-3380-4b17-b8e8-7c7545a6f1bd">
normalise = True
# </halerium id="9330ed5e-3380-4b17-b8e8-7c7545a6f1bd">

if normalise:
    df = Transformer.normalise(df)
df

Binning

In [0]:
# <halerium id="9330ed5e-3380-4b17-b8e8-7c7545a6f1bd">
binning_columns = ['meantemp']
binning_thresholds = [0, 0.33, .67, 1]
binning_labels = ["Low", "Medium", "High"] # Labels must be 1 fewer than thresholds
# </halerium id="9330ed5e-3380-4b17-b8e8-7c7545a6f1bd">

df = Transformer.binning(df, binning_columns, binning_thresholds, binning_labels)
df

Select Columns

In [0]:
# <halerium id="9330ed5e-3380-4b17-b8e8-7c7545a6f1bd">
select_columns = ['meantemp', 'humidity', 'windspeed', 'meanpressure', 'result', 'meantemp_bin']
# </halerium id="9330ed5e-3380-4b17-b8e8-7c7545a6f1bd">

df = df[select_columns]
df

## 6. Export the prepared data

View final df before export

In [0]:
df

In [0]:
Plotter.plot_line(df)

In [0]:
# <halerium id="c9f23d08-4d4f-44d1-ae91-6bb801a365ec">
export_path = './export.csv'
# </halerium id="c9f23d08-4d4f-44d1-ae91-6bb801a365ec">

df.to_csv(export_path)